# Instalação dos requisitos

In [1]:
!pip3 install --upgrade webvtt-py

Requirement already up-to-date: webvtt-py in /opt/conda/lib/python3.8/site-packages (0.4.5)


# Código

In [2]:
import os
import webvtt
import sqlite3
import json
import re

diretorio_dados = f'{os.path.abspath("../dados")}{os.path.sep}'
diretorio_legendas = f'{os.path.abspath("../dados/legendas")}{os.path.sep}'
legendas_baixadas = os.listdir(diretorio_legendas)


In [3]:
def carregar_dados_playlist():
    with open(f'{diretorio_dados}playlist.json', 'r') as arquivo:
        return json.load(arquivo)


In [4]:
def carregar_legendas(conexao, legendas, dados):
    resumo = {dado['id']: dado['title']
              for dado in dados['entries'] if dado['title'] != '[Private video]'}
    dados_insercao = list()
    ids_existentes = [id_youtube[0] for id_youtube in conexao.execute(
        'select id_youtube from episodio').fetchall()]

    for legenda in legendas:
        id_video = legenda.split('.')[0]

        if not id_video in ids_existentes:
            titulo = resumo[id_video]
            resultado = re.findall(r'(#\s*)(\d+)', titulo)
            if len(resultado) > 0:
                numero_episodio = re.findall(r'(#\s*)(\d+)', titulo)[0][1]
                conexao.execute('insert into episodio values (?, ?, ?)',
                                (numero_episodio, id_video, titulo))

                for conteudo in webvtt.read(os.path.join(diretorio_legendas, legenda)):
                    conexao.execute('insert into legenda values (?, ?, ?, ?, ?)', (
                        None, numero_episodio, conteudo.start, conteudo.end, conteudo.text))

    conexao.commit()


In [5]:
def executar():
    with sqlite3.connect(f'{diretorio_dados}dados.db') as conexao:
        conexao.execute('''
        create table if not exists episodio (
            id_episodio integer primary key not null,
            id_youtube text not null,
            titulo text not null
        );
        ''')

        conexao.execute('''        
        create table if not exists legenda (
            id_legenda integer primary key autoincrement not null,
            id_episodio integer not null,
            inicio time not null,
            fim time not null,
            conteudo text not null,
            foreign key(id_episodio) references episodio(id_episodio)
        );
        ''')

        dados = carregar_dados_playlist()
        carregar_legendas(conexao, legendas_baixadas, dados)
        print('Processo concluído')


In [6]:
executar()

Processo concluído


# Referências

[sqlite3 — DB-API 2.0 interface for SQLite databases](https://docs.python.org/3/library/sqlite3.html)

[Datatypes In SQLite Version 3](https://sqlite.org/datatype3.html)